In [6]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import datetime


In [3]:
# load the dataframe with capitals
df = pd.read_csv("concap.csv")

# rename so that the column names are shorter and comply with PEP-8
df.rename(columns={"CountryName": "Country", "CapitalName": "capital", "CapitalLatitude": "lat", "CapitalLongitude": "lon", "CountryCode": "code", "ContinentName": "continent"}, inplace=True)
df.sample(3)


,Country,capital,lat,lon,code,continent
124,Latvia,Riga,56.950000,24.100000,LV,Europe
53,Cocos Islands,West Island,-12.166667,96.833333,CC,Australia
56,Democratic Republic of the Congo,Kinshasa,-4.316667,15.300000,CD,Africa


In [4]:
# to start with let's filter only 2 capitals. Rome and Paris.
ropa = df[df["capital"].isin(["Rome","Paris"])].reset_index()
ropa

,index,Country,capital,lat,lon,code,continent
0,81,France,Paris,48.866667,2.333333,FR,Europe
1,110,Italy,Rome,41.900000,12.483333,IT,Europe


In [2]:
lat_1 = '47.4254570232' # Hochbauamt
lon_1 = '9.37577578175'
lat_2 = '47.4249702' # Liegenschaften
lon_2 = '9.3730005'


In [3]:
# call the OSMR API
r = requests.get(f"http://router.project-osrm.org/route/v1/foot/{lon_1},{lat_1};{lon_2},{lat_2}?overview=false""")

In [4]:
routes = json.loads(r.content)
route_1 = routes.get("routes")[0]

In [7]:
str(datetime.timedelta(seconds=route_1["duration"]))

'0:01:11.800000'

In [16]:
import datetime

In [5]:
r = requests.get('https://daten.stadt.sg.ch/api/records/1.0/search/?dataset=freie-parkplatze-in-der-stadt-stgallen-pls&q=&geofilter.distance=47.42427%2C9.37186%2C70')

In [17]:
df = json_normalize(json.loads(r.content)['records'])

In [21]:
df[['lat','lon']] = df['geometry.coordinates'].tolist()

In [22]:
df

,datasetid,recordid,record_timestamp,fields.zeitpunkt,fields.belegung_prozent,fields.phid,fields.shortmax,fields.shortfree,fields.shortoccupied,fields.standort,fields.phstate,fields.dist,fields.phname,geometry.type,geometry.coordinates,lat,lon
0,freie-parkplatze-in-der-stadt-stgallen-pls,6e76a2a3d268993aaf5a705f865891d39303d0b3,2021-10-22T07:49:00.368000+00:00,2021-10-22T07:46:37+00:00,88,P22,84,10,74,"[47.424453, 9.371221]",offen,52.2025071449,Rathaus,Point,"[9.371221, 47.424453]",9.371221,47.424453


In [17]:
immo = pd.read_csv('https://raw.githubusercontent.com/cividi/st-gallen-urban-indicators/main/data/price-monitoring/price-monitoring.csv',usecols=['id','rent','lat','lng','street_number'])

In [18]:
immo.head()

,id,rent,street_number,lat,lng
0,108761792,1745,Berneggstrasse 28,47.420898,9.376225
1,2147994348,1720,Heiligkreuzstrasse 53,47.440407,9.393985
2,3000166218,1150,Lindenstrasse 76,47.433056,9.393829
3,3000251780,1390,Feldlistrasse 11,47.421700,9.356135
4,3000402506,2130,Goliathgasse 37,47.427975,9.377123


In [19]:
immo = immo[~immo.lat.isna()]

In [23]:
unisg = ['47.432277', '9.375572']

In [24]:
unisg[1]

'9.375572'

In [49]:
# call the OSMR API
r = json.loads(requests.get(f"http://router.project-osrm.org/route/v1/foot/{immo.iloc[0].lng},{immo.iloc[0].lat};{unisg[1]},{unisg[0]}?overview=false""").content).get("routes")[0]

In [54]:
r['duration']

2022.6

In [62]:
dist = []
dur = []
for index,row in immo.iterrows():
    r = json.loads(requests.get(f"http://router.project-osrm.org/route/v1/foot/{row.lng},{row.lat};{unisg[1]},{unisg[0]}?overview=false""").content).get("routes")[0]
    dist.append(r['distance'])
    dur.append(r['duration'])


In [73]:
immo.insert(loc=len(immo.columns), column='distance_meters',value=dist)

In [75]:
immo.count()

id                  485
rent                485
street_number       485
lat                 485
lng                 485
duration            485
duration_seconds    485
distance_meters     485
dtype: int64

In [71]:
immo.drop(columns='distance_meters',inplace=True)

In [59]:
immo.rename(columns = {'distance':'duration'}, inplace=True)

In [74]:
immo.to_csv('immo_uni.csv')

In [77]:
immo[immo.distance_meters <= 2000]

,id,rent,street_number,lat,lng,duration,duration_seconds,distance_meters
2,3000166218,1150,Lindenstrasse 76,47.433056,9.393829,0:04:09.600000,249.6,1870.4
4,3000402506,2130,Goliathgasse 37,47.427975,9.377123,0:02:56.700000,176.7,1433.4
9,3000836173,1090,Langgasse 89,47.440033,9.391563,0:03:33.400000,213.4,1702.3
11,3000872761,1180,Linsebühlstrasse 16,47.424324,9.380756,0:03:17.200000,197.2,1792.9
12,3000873636,1640,Ludwigstr. 21,47.441250,9.378373,0:03:18.600000,198.6,1647.7
...,...,...,...,...,...,...,...,...
525,3001451526,940,Engelgasse 8,47.426357,9.374825,0:03:59.600000,239.6,1607.2
528,3001454103,1180,Langgasse 46,47.437061,9.388271,0:02:55.800000,175.8,1286.1
529,3001454127,1590,Langgasse 46,47.437061,9.388271,0:02:55.800000,175.8,1286.1
544,3001465599,1240,Moosbruggstrasse 5,47.423676,9.379521,0:03:21.600000,201.6,1823.4
